# 🚀 **Capstone Project Curso Desarrolador 10x de Instituto de Inteligencia Artificial - Entregable 2**

## 📄 Información del Proyecto
**Estudiante:** Araceli Fradejas Muñoz  
**Curso:** Curso Desarrollador 10x – Instituto de Inteligencia Artificial  
**Fecha:** 21/04/2025  

---

## 📝 Descripción del Proyecto: Asistente de IA para Call Center

### 🔎 Contexto

**KelceTS S.L.** es una startup ficticia especializada en la venta online de zapatillas en toda Europa. El área de atención al cliente recibe diariamente comentarios de usuarios en múltiples idiomas europeos a través de distintos canales (redes sociales, email, ...), los cuáles deben ser valorados manualmente por agentes del Call Center para:

- Analizar posibles problemas logísticos o de calidad
- Responder de forma personalizada al cliente
- Notificar incidencias a equipos internos o a proveedores

Este proceso manual es lento, poco homogéneo y costoso.

---

## 🎯 Objetivo del Notebook

Diseñar una **herramienta basada en IA generativa** accesible desde el navegador, que permita a un agente de Call Center:

- 🗣️ Introducir un comentario de cliente o elegir un ejemplo realista
- 🌍 Detectar automáticamente el idioma y traducirlo si es necesario
- ✅ Evaluar el comentario aplicando reglas internas de calidad y logística
- 💌 Generar automáticamente:
  - Emails personalizados para el cliente (en su idioma original)
  - Notificaciones internas para los equipos de logística y calidad
  - Comunicaciones formales para proveedores externos

El sistema está diseñado para funcionar principalmente con **OpenAI**, y cuenta con un **fallback automático con Gemini (Google Cloud)** en caso de fallo o límite de uso, garantizando así una experiencia robusta y sin interrupciones.

---

## 🛠️ Tecnologías Utilizadas

- **Python** en Google Colab como entorno de desarrollo
- **OpenAI (GPT-3.5/4)** como motor principal de análisis y generación
- **Gemini Pro (Google Cloud)** como motor de respaldo ante errores o límites
- **Gradio** para crear una interfaz web interactiva y accesible desde el navegador
- **Langdetect** para identificar el idioma original del comentario
- **Pandas + HTML + Markdown** para mostrar resultados legibles y exportables

---

## 💡 Impacto Esperado

Esta herramienta permitirá al equipo del Call Center de KelceTS:

- ⏱️ Reducir el tiempo medio de gestión de comentarios
- 🌍 Responder a clientes en cualquiera de los 24 idiomas oficiales de la Unión Europea
- 📦 Detectar patrones recurrentes de fallos en productos o incidencias de envío
- 📊 Exportar informes para auditoría y mejora continua del servicio

El objetivo final es **escalar la atención al cliente de forma automatizada, multilingüe y personalizada**, manteniendo los estándares de calidad de la marca.


## **1. ⚙️ Preparación del entorno**

En este apartado preparamos el entorno de ejecución e instalamos las librerías necesarias para desplegar el asistente de IA en forma de aplicación web interactiva, accesible desde el navegador gracias a **Gradio**.

Esta herramienta está pensada como una solución práctica para **agentes del Call Center de KelceTS S.L.**, que necesitan:

- Analizar automáticamente comentarios de clientes en múltiples idiomas
- Generar comunicaciones personalizadas y notificaciones internas
- Visualizar y exportar fácilmente los resultados

Además, conectamos el notebook con los datos disponibles en el repositorio de GitHub, en el directorio `data`, sin depender de Google Drive (en versiones anteriores realizamos esta dependencia y aqúi está optimizado para que cualquier usuario pueda ejecutarlo).

Los archivos que se utilizarán están alojados en la carpeta `data` del repositorio público:

- `BD Comentarios KelceTS.txt`
- `Reglas de como valorar un comentario KelceTS S.L..xlsx`
- `Reglas de calidad clientes KelceTS S.L..xlsx`
- `Reglas de comunicaciones equipos calidad y logística KelceTS S.L..xlsx`
- `Reglas de medidas de calidad KelceTS S.L..xlsx`


 🚧 En las siguientes celdas se instalan todas las dependencias necesarias y se configura el entorno base para comenzar con el análisis automatizado.

- 📡 Conexión con modelos de lenguaje: **OpenAI** y **Gemini**
- 🌍 Detección automática de idioma: **langdetect**
- 🔐 Carga segura de claves: **dotenv**
- 🧠 Interfaz sencilla para agentes: **Gradio**


> ⚠️ Ejecuta esta celda antes de continuar con la carga de datos o modelos IA


In [3]:
# Instalación de librerías mínimas necesarias
!pip install -q openai gradio google-generativeai python-dotenv langdetect

# Importaciones del sistema
import os
import json
import re
import time
import random
from datetime import datetime

# Detección de idioma
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # Para resultados reproducibles

# Modelos de lenguaje
import openai
import google.generativeai as genai

# Interfaz web
import gradio as gr

# Estilo y reproducibilidad
random.seed(42)


### ***1.1 🔐 Carga de claves de OpenAI y Gemini desde `.env` o desde Secrets en Colab***

Este notebook se ejecuta de forma segura y privada gracias a un sistema mixto de carga de claves:

1. Primero intenta cargar un archivo `.env` en la raíz del proyecto, que debe contener las variables:

   - `OPENAI_API_KEY`
   - `GEMINI_API_KEY`

2. Si el archivo `.env` no está disponible (por ejemplo, en Google Colab), intentará recuperar las claves desde **Colab Secrets**, accediendo a `userdata.get()`.

Este enfoque garantiza:
- ✅ Seguridad (las claves no se exponen directamente en el código)
- ✅ Compatibilidad con notebooks públicos en GitHub o compartidos por Colab

---

⚠️ **MUY IMPORTANTE**  
- Si estás ejecutando este notebook por primera vez, asegúrate de tener un archivo `.env` con tus claves.
- 🔑 También puedes guardarlas de forma segura en Colab usando el icono 🔐 (llave) que aparece a la izquierda.
- ❌ Si no se encuentran las claves, las llamadas a la API fallarán en las siguientes celdas.


In [4]:
# Configuración de claves OpenAI + Gemini (.env o Colab Secrets)

import os
from dotenv import load_dotenv

# 1️. Intentamos cargar desde archivo .env si existe
if os.path.exists(".env"):
    load_dotenv()
    print("📄 Archivo .env cargado correctamente.")
else:
    print("⚠️ No se encontró archivo .env. Intentando cargar desde Colab Secrets...")
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
        os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
        print("🔐 Claves cargadas desde Colab Secrets.")
    except Exception as e:
        print(f"❌ No se pudieron obtener claves desde Colab Secrets: {str(e)}")

# 2️. Configuramos los clientes modernos
from openai import OpenAI
import google.generativeai as genai

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# 3️. Validación de claves
if not os.getenv("OPENAI_API_KEY") or not os.getenv("GEMINI_API_KEY"):
    raise ValueError("❌ No se detectaron las claves API necesarias. Por favor, configura tu archivo .env o Colab Secrets.")
else:
    print("✅ Claves API configuradas correctamente.")


⚠️ No se encontró archivo .env. Intentando cargar desde Colab Secrets...
🔐 Claves cargadas desde Colab Secrets.
✅ Claves API configuradas correctamente.


### ***1.2 ⚙️ Motor IA con fallback automático (OpenAI + Gemini)***

Una de las prioridades clave en el diseño de esta herramienta para el Call Center de **KelceTS S.L.** es garantizar la **disponibilidad continua del servicio de análisis**, incluso en situaciones donde puedan fallar los proveedores de IA (por ejemplo, por cuotas agotadas, errores temporales o caídas del servicio).

Por eso, en esta implementación se ha incluido una función central llamada `call_ai_model()`, que permite:

- ✅ Usar **OpenAI** como modelo principal (por su precisión en la comprensión del lenguaje natural)
- 🔄 Hacer **fallback automático a Gemini (Google Cloud)** en caso de fallo o no disponibilidad
- 📡 Garantizar la ejecución de la herramienta sin interrupciones para el agente del Call Center

> Este enfoque evita errores inesperados y asegura que siempre se genere una respuesta al cliente, **sin depender de un único proveedor de IA**.

Esta función puede utilizarse de forma unificada en todo el flujo de la aplicación Gradio, lo que mejora la mantenibilidad del código y asegura resiliencia operativa.


In [5]:
# Función con fallback automático entre OpenAI y Gemini

def call_ai_model(prompt, system_message="Eres un asistente de atención al cliente para Kelce TS tienda de zapatillas.", model="gpt-3.5-turbo"):
    """
    Ejecuta un prompt con OpenAI como primera opción.
    Si falla, hace fallback a Gemini.
    """
    try:
        print("🔵 Llamando a OpenAI...")
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        content = response.choices[0].message.content.strip()
        print("✅ Respuesta generada por OpenAI.")
        return content

    except Exception as e_openai:
        print(f"⚠️ OpenAI falló: {e_openai}")
        print("🟡 Cambiando a Gemini...")

        try:
            gemini_response = genai.GenerativeModel("gemini-pro").generate_content(prompt)
            content = gemini_response.text.strip()
            print("✅ Respuesta generada por Gemini.")
            return content
        except Exception as e_gemini:
            print(f"❌ Gemini también falló: {e_gemini}")
            return "⚠️ No se pudo generar una respuesta en este momento. Inténtalo de nuevo más tarde."


### ***1.3 🔄 Traducción automática y validación centralizada***

Una parte fundamental de esta herramienta para el Call Center de **KelceTS S.L.** es la capacidad de **responder automáticamente a los clientes en su idioma original**, garantizando que:

- 🗣️ La traducción generada es fiel, clara y contextual
- 🔁 No se pierda información clave al traducir comentarios críticos
- ⚡ Se acelere el flujo de trabajo sin errores humanos ni retraducciones

Para ello, este sistema centraliza todo el flujo de traducción y validación en funciones especializadas:

- `clear_cache()`: limpia la caché y memoria antes de cada nuevo análisis
- `translate_comment()`: traduce el comentario con `OpenAI` y cachea el resultado
- `validate_translation()`: verifica que la traducción es proporcional y que mantiene conceptos clave
- `process_comment()`: ejecuta el proceso completo en un solo paso (ideal para Gradio)

Este enfoque no solo mejora la eficiencia, sino que garantiza la **consistencia en la comunicación multilingüe** que ofrece la marca.


In [6]:
# Utilidades para traducción y validación centralizadas

import gc
from functools import lru_cache

# 1. Forzar limpieza de memoria y caché
def clear_cache():
    gc.collect()
    try:
        translate_comment.cache_clear()
        validate_translation.cache_clear()
    except NameError:
        pass

# Traducción automática usando OpenAI (fallback ya lo tienes en call_ai_model si quieres integrarlo)
@lru_cache(maxsize=128)
def translate_comment(comment: str, source_lang: str = None, target_lang: str = "es") -> str:
    prompt = f"Traduce este texto al {target_lang}:\n\n{comment}"
    if source_lang:
        prompt = f"El texto está en {source_lang}. " + prompt

    return call_ai_model(prompt, system_message="Actúa como traductor profesional.", model="gpt-3.5-turbo")

# Validación de que la traducción tiene sentido
@lru_cache(maxsize=128)
def validate_translation(original: str, translated: str) -> bool:
    orig_len = len(original.split())
    trans_len = len(translated.split())

    # 30% de margen superior o inferior
    if trans_len < 0.7 * orig_len or trans_len > 1.3 * orig_len:
        return False

    # Palabras relevantes en mayúscula que no aparezcan traducidas
    for token in original.split():
        if token.istitle() and token.lower() not in translated.lower():
            return False

    return True

# Proceso completo para una sola entrada
def process_comment(comment: str, source_lang: str = None):
    clear_cache()
    translation = translate_comment(comment, source_lang)
    is_valid = validate_translation(comment, translation)
    return {
        "original": comment,
        "translated": translation,
        "translation_valid": is_valid
    }


### ***1.4 📁 Carga de archivos del proyecto desde GitHub***

Para asegurar que la aplicación Gradio del Call Center de **KelceTS S.L.** pueda acceder a los comentarios de clientes y a todas las reglas internas necesarias para el análisis, clonamos directamente el repositorio del proyecto desde GitHub.

Esto nos permite trabajar con archivos reales (`.txt`, `.xlsx`) sin depender de Google Drive, garantizando:

- ✅ Reproducibilidad del proyecto en cualquier entorno
- ✅ Independencia de sesiones personales de Colab
- 📁 Archivos cargados automáticamente desde la carpeta `/data` del repositorio

Los archivos utilizados incluyen:
- `BD Comentarios KelceTS.txt`
- `Reglas de cómo valorar un comentario KelceTS SL.xlsx`
- `Reglas de calidad clientes KelceTS SL.xlsx`
- `Reglas de comunicaciones equipos calidad y logística KelceTS SL.xlsx`
- `Reglas de medidas de calidad KelceTS SL.xlsx`


In [7]:
# Clonamos el repositorio de GitHub con los archivos del proyecto
!git clone https://github.com/AraceliFradejas/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas.git

#  Definimos la ruta base donde están los datos
ruta_base = "/content/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/data"
archivo_comentarios = f"{ruta_base}/BD Comentarios KelceTS.txt"

# Verificamos los archivos disponibles en la carpeta 'data'
print("📁 Archivos encontrados en la carpeta 'data':")
!ls -1 {ruta_base}


Cloning into 'Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 84 (delta 46), reused 44 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (84/84), 1.42 MiB | 3.76 MiB/s, done.
Resolving deltas: 100% (46/46), done.
📁 Archivos encontrados en la carpeta 'data':
'BD Comentarios KelceTS.txt'
'Kelce TS LOGO.png'
'Reglas de calidad clientes KelceTS SL.xlsx'
'Reglas de como valorar un comentario KelceTS SL.xlsx'
'Reglas de comunicaciones equipos calidad y logistica KelceTS SL.xlsx'
'Reglas de medidas de calidad KelceTS SL.xlsx'


## **2. 🌍 Detección del idioma del comentario**

Antes de traducir o responder un comentario, es necesario identificar el idioma original para poder:

- Evitar traducciones innecesarias si el texto ya está en español
- Mostrar el idioma de forma explícita en el análisis
- Utilizar el mismo idioma en las respuestas al cliente

Para ello se usa la librería `langdetect`, que permite detectar automáticamente más de 25 idiomas europeos. Se ha configurado con una semilla fija para garantizar reproducibilidad.


In [8]:
# Detección de idioma

from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # Para resultados consistentes

def detectar_idioma(texto: str) -> str:
    """
    Detecta el idioma del texto y devuelve su nombre en español.
    """
    try:
        codigo_idioma = detect(texto)
        mapa_idiomas = {
            'bg': 'búlgaro', 'hr': 'croata', 'cs': 'checo', 'da': 'danés', 'nl': 'neerlandés',
            'en': 'inglés', 'et': 'estonio', 'fi': 'finlandés', 'fr': 'francés', 'de': 'alemán',
            'el': 'griego', 'hu': 'húngaro', 'ga': 'irlandés', 'it': 'italiano', 'lv': 'letón',
            'lt': 'lituano', 'mt': 'maltés', 'pl': 'polaco', 'pt': 'portugués', 'ro': 'rumano',
            'sk': 'eslovaco', 'sl': 'esloveno', 'es': 'español', 'sv': 'sueco',
            'ca': 'catalán', 'gl': 'gallego', 'eu': 'euskera', 'no': 'noruego', 'is': 'islandés', 'tr': 'turco'
        }
        return mapa_idiomas.get(codigo_idioma, codigo_idioma)
    except Exception as e:
        print(f"Error en detección de idioma: {str(e)}")
        return "desconocido"


## **3. 🧪 Comentarios de ejemplo para test inicial**

Incluimos una lista de comentarios predefinidos y sus respuestas simuladas para facilitar la demostración de la aplicación Gradio paras los usuarios que van a trabajar con dichas aplicación en el call center (sin depender de la API).

Esto permite a los agentes del Call Center:
- Analizar ejemplos reales sin coste
- Ver resultados inmediatos
- Probar distintos idiomas y tipos de incidencias para aprender cómo trabajar con la aplicación.


In [9]:
# Definimos una lista de comentarios de ejemplo garantizados
EJEMPLOS_COMENTARIOS = [
    "Als die Schuhe angekommen sind habe ich mich erstmal sehr gefreut und hätte wahrscheinlich 5 Sterne vergeben! Sie haben gut gepasst, wirkten auf den ersten Blick relativ gut verarbeitet und sehr bequem. Leider musste ich fast sofort feststellen dass die Schuhe sich unnormal schnell abnutzen.",

    "Buenos días, El día 23 de febrero realiza el pedido y llego el día 25. Las estrene al cabo de dos días y en pocos días de uso están así. Ver fotos. El trato es de ir por la calle de paseo. Son las segundas que utilizo. las primeras estuve muy contento. en cambio con estas no me han durado ni un mes de uso.",

    "Lähetys nopea, hyvin pakattu kuten kuvasta näkyy, näyttävät olevan hyvin liimattuja ja niissä on vahvistettu ommel. Kooltaan tilasin 43, jalkani on 27,5 cm pitkä. Ne sopivat minulle täydellisesti, eivät purista varpaitani eivätkä ole liian suuret.",

    "Son bastante cómodas para salir a dar una vuelta, pero para usarlas para pasear durante una semana cansan un poco porque no tienen amortiguación y al final te cansas. Para salidas ocasionales son perfectas. Muy buen producto.",

    "Son un poco plasticosas por fuera. Son bonitas. Me quedan algo grandes, el 46, quizás sea el problema de usar 45 y medio. Con plantillas se mitiga algo el problema. No las devuelvo porque se pueden usar así, pero quizás hubiera sido mejor comprar unas que me pueda probar bien primero."
]

# Respuestas predefinidas para cada ejemplo (simulación)
RESPUESTAS_SIMULADAS = [
    # Comentario 1 (alemán, negativo por materiales)
    {
        "analisis": {
            "idioma": "alemán",
            "envio_96h": "no mencionado",
            "embalaje_danado": "no mencionado",
            "talla_correcta": "sí",
            "materiales_calidad": "no",
            "tipo_uso": "diario, paseos por ciudad",
            "cumple_expectativas": "no"
        },
        "valoracion": "negativa",
        "comunicaciones": {
            "email_cliente": "Lieber Kunde,\n\nVielen Dank für dein Feedback. Es tut uns sehr leid zu hören, dass sich die Schuhe so schnell abgenutzt haben.\n\nWir nehmen deine Rückmeldung sehr ernst und möchten dir als Entschädigung einen Rabatt von 25% auf deinen nächsten Einkauf sowie kostenlosen Versand anbieten. Außerdem wird ein Mitglied unseres Teams die beschädigten Schuhe innerhalb der nächsten 72 Stunden bei dir abholen. Bitte halte die Schuhe in der Originalverpackung oder einer ähnlichen Verpackung bereit.\n\nBei weiteren Fragen stehen wir dir gerne unter atencionalcliente@kelcetssl.com zur Verfügung.\n\nMit freundlichen Grüßen,\nKelceTS Team",
            "email_cliente_traduccion": "Estimado cliente,\n\nGracias por tu comentario. Lamentamos mucho saber que los zapatos se han desgastado tan rápidamente.\n\nNos tomamos muy en serio tu feedback y queremos ofrecerte un 25% de descuento en tu próxima compra, así como envío gratuito. Además, un miembro de nuestro equipo recogerá los zapatos dañados dentro de las próximas 72 horas. Por favor, ten los zapatos listos en su embalaje original o uno similar.\n\nSi tienes más preguntas, no dudes en contactarnos en atencionalcliente@kelcetssl.com.\n\nSaludos cordiales,\nKelceTS Team",
            "notificacion_interna": "Hola equipo,\n\nHemos recibido un comentario negativo de un cliente alemán sobre la calidad de los materiales de nuestras zapatillas. El cliente menciona que después de 2,5 meses de uso casi diario, las zapatillas presentan un desgaste anormal, con zonas interiores desgastadas, agujeros en la suela y desprendimiento del cuero sintético.\n\nSe requiere contactar al proveedor en las próximas 24 horas para analizar este problema de calidad y establecer un plan de acción para mejorar la durabilidad de los materiales.\n\nAsistente IA de KelceTS S.L.",
            "email_proveedor": "Estimado proveedor,\n\nNos ponemos en contacto con usted en relación a un problema de calidad identificado en nuestras zapatillas. Hemos recibido quejas de clientes indicando que después de aproximadamente 2,5 meses de uso diario, las zapatillas presentan un desgaste anormal: desgaste interior, agujeros en la suela y desprendimiento del cuero sintético.\n\nSolicitamos que analicen este caso en un plazo de 24 horas y nos propongan un plan de acción para mejorar la durabilidad de los materiales en futuros lotes de producción.\n\nAtentamente,\n\nRodrigo Clemente\nDirector de Logística de KelceTS S.L."
        }
    },
    # Comentario 2 (español, negativo por materiales)
    {
        "analisis": {
            "idioma": "español",
            "envio_96h": "sí",
            "embalaje_danado": "no mencionado",
            "talla_correcta": "no mencionado",
            "materiales_calidad": "no",
            "tipo_uso": "paseo diario",
            "cumple_expectativas": "no"
        },
        "valoracion": "negativa",
        "comunicaciones": {
            "email_cliente": "Estimado amigo,\n\nGracias por compartir tu experiencia con nosotros. Lamentamos mucho que las zapatillas hayan presentado problemas de desgaste en tan poco tiempo de uso.\n\nRegistramos este punto de mejora y como medida de calidad por las molestias te regalamos un 25% de descuento en tu próxima compra, que además no tendrá gastos de envío. También vamos a enviar a un miembro del staff en menos de 72h para recoger el par de zapatillas defectuosas, por lo que deberás tenerlas preparadas dentro del embalaje original o en uno similar (si ya no lo tienes).\n\nSi tienes alguna otra pregunta o inquietud, no dudes en contactarnos en atencionalcliente@kelcetssl.com.\n\nUn cordial saludo,\nKelceTS Team",
            "email_cliente_traduccion": "",
            "notificacion_interna": "Hola equipo,\n\nHemos recibido un comentario negativo de un cliente sobre la calidad de los materiales de nuestras zapatillas. El cliente menciona que después de muy pocos días de uso, las zapatillas muestran signos de desgaste, específicamente en la zapatilla derecha en la zona interior lateral y con el talón hundido, lo que las hace incómodas para caminar.\n\nSe requiere contactar al proveedor en las próximas 24 horas para analizar este problema de calidad y establecer un plan de acción para mejorar la durabilidad de los materiales.\n\nAsistente IA de KelceTS S.L.",
            "email_proveedor": "Estimado proveedor,\n\nNos ponemos en contacto con usted en relación a un problema de calidad identificado en nuestras zapatillas. Hemos recibido quejas de clientes indicando que después de muy pocos días de uso normal para paseo, las zapatillas presentan desgaste en la zona interior lateral y hundimiento del talón, lo que las hace incómodas para caminar.\n\nSolicitamos que analicen este caso en un plazo de 24 horas y nos propongan un plan de acción para mejorar la durabilidad de los materiales en futuros lotes de producción.\n\nAtentamente,\n\nRodrigo Clemente\nDirector de Logística de KelceTS S.L."
        }
    },
    # Comentario 3 (finlandés, positivo)
    {
        "analisis": {
            "idioma": "finlandés",
            "envio_96h": "sí",
            "embalaje_danado": "no",
            "talla_correcta": "sí",
            "materiales_calidad": "sí",
            "tipo_uso": "no mencionado",
            "cumple_expectativas": "sí"
        },
        "valoracion": "positiva",
        "comunicaciones": {
            "email_cliente": "Hyvä asiakas,\n\nKiitos palautteestasi! Olemme iloisia kuullessamme, että olet tyytyväinen kenkiisi ja että ne sopivat sinulle täydellisesti.\n\nJos sinulla on kysyttävää tai tarvitset apua, älä epäröi ottaa meihin yhteyttä osoitteessa atencionalcliente@kelcetssl.com.\n\nYstävällisin terveisin,\nKelceTS Team",
            "email_cliente_traduccion": "Estimado cliente,\n\n¡Gracias por tu comentario! Nos alegra saber que estás satisfecho con tus zapatos y que te quedan perfectamente.\n\nSi tienes alguna pregunta o necesitas ayuda, no dudes en contactarnos en atencionalcliente@kelcetssl.com.\n\nSaludos cordiales,\nKelceTS Team",
            "notificacion_interna": "",
            "email_proveedor": ""
        }
    },
    # Comentario 4 (español, parcialmente negativo por amortiguación)
    {
        "analisis": {
            "idioma": "español",
            "envio_96h": "no mencionado",
            "embalaje_danado": "no mencionado",
            "talla_correcta": "no mencionado",
            "materiales_calidad": "parcialmente",
            "tipo_uso": "salidas ocasionales y paseos largos",
            "cumple_expectativas": "parcialmente"
        },
        "valoracion": "neutra",
        "comunicaciones": {
            "email_cliente": "Estimado amigo,\n\nMuchas gracias por tomarte el tiempo para compartir tu experiencia con nuestras zapatillas. Nos alegra saber que las encuentras cómodas para salidas ocasionales.\n\nHemos tomado nota sobre tu comentario acerca de la falta de amortiguación para paseos más largos. Este tipo de feedback es muy valioso para nosotros, ya que nos ayuda a mejorar constantemente nuestros productos.\n\nSi tienes alguna otra pregunta o inquietud, no dudes en contactarnos en atencionalcliente@kelcetssl.com.\n\nUn cordial saludo,\nKelceTS Team",
            "email_cliente_traduccion": "",
            "notificacion_interna": "Hola equipo,\n\nHemos recibido un comentario mixto de un cliente sobre nuestras zapatillas. El cliente está satisfecho con el producto para salidas ocasionales, pero menciona que la falta de amortiguación hace que resulten cansadas para paseos largos de una semana.\n\nSugerimos evaluar la posibilidad de mejorar la amortiguación en futuros modelos para aumentar la comodidad en uso prolongado.\n\nAsistente IA de KelceTS S.L.",
            "email_proveedor": ""
        }
    },
    # Comentario 5 (español, negativo por talla)
    {
        "analisis": {
            "idioma": "español",
            "envio_96h": "no mencionado",
            "embalaje_danado": "no mencionado",
            "talla_correcta": "no",
            "materiales_calidad": "parcialmente",
            "tipo_uso": "no mencionado",
            "cumple_expectativas": "parcialmente"
        },
        "valoracion": "negativa",
        "comunicaciones": {
            "email_cliente": "Estimado amigo,\n\nGracias por compartir tu experiencia con nuestras zapatillas. Lamentamos que la talla no sea la adecuada para ti y que los materiales no cumplan completamente con tus expectativas.\n\nRegistramos este punto de mejora y como medida de calidad por las molestias vamos a enviarte en un plazo inferior a 72h un par de zapatillas de la talla correcta (45.5) sin coste alguno. Por favor, debes tener preparado el par de zapatillas que no te valen dentro del embalaje original o en uno similar (si ya no lo tienes).\n\nSi tienes alguna otra pregunta o inquietud, no dudes en contactarnos en atencionalcliente@kelcetssl.com.\n\nUn cordial saludo,\nKelceTS Team",
            "email_cliente_traduccion": "",
            "notificacion_interna": "Hola equipo,\n\nHemos recibido un comentario de un cliente que indica problemas con la talla de las zapatillas (talla 46 le queda grande, normalmente usa 45.5) y también menciona que el material le parece algo \"plasticoso\".\n\nSe requiere contactar al departamento de logística para enviar un nuevo par en talla 45.5 al cliente en menos de 48 horas, y coordinar la recogida del par actual.\n\nAsistente IA de KelceTS S.L.",
            "email_proveedor": "Estimado proveedor,\n\nNos ponemos en contacto con usted en relación a un problema identificado con nuestras zapatillas. Un cliente ha reportado que la talla 46 le queda demasiado grande cuando normalmente usa talla 45.5, y también ha mencionado que el material le parece algo \"plasticoso\".\n\nSolicitamos que envíen un nuevo par de zapatillas en talla 45.5 al domicilio del cliente en menos de 48 horas y, al entregárselas, recojan para devolvernos el par que no es de la talla correcta.\n\nAtentamente,\n\nRodrigo Clemente\nDirector de Logística de KelceTS S.L."
        }
    }
]

## **4. 🖼️ Generación visual de resultados en HTML**

La función `generar_html_resultado()` convierte los resultados del análisis y las comunicaciones generadas en una presentación visual, clara y estructurada para el usuario de la aplicación en el call center.

Esta función:
- Organiza la información por secciones (análisis, emails, notificaciones…)
- Usa estilos para mejorar la legibilidad
- Se adapta perfectamente al entorno web de Gradio


In [10]:
from datetime import datetime

def generar_html_resultado(resultado, comentario_original):
    """Genera HTML para mostrar el resultado del análisis"""
    try:
        analisis = resultado["analisis"]
        valoracion = resultado["valoracion"]
        comunicaciones = resultado["comunicaciones"]
        traduccion_comentario = resultado.get("traduccion_comentario", "")

        color_valoracion = {
            "positiva": "#27ae60",
            "negativa": "#e74c3c",
            "neutra": "#f39c12",
            "error": "#7f8c8d"
        }.get(valoracion, "#7f8c8d")

        comentario_html = f"""
        <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
            <h3 style="margin-top: 0; color: #2c3e50;">Comentario Original</h3>
            <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif;">
                {comentario_original}
            </div>
        """

        if traduccion_comentario and analisis["idioma"] != "español":
            comentario_html += f"""
            <h4 style="margin-top: 15px; color: #2c3e50;">Traducción al Español:</h4>
            <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif; border-left: 3px solid #e31837;">
                {traduccion_comentario}
            </div>
            """

        comentario_html += "</div>"

        analisis_html = f"""
        <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
            <h3 style="margin-top: 0; color: #2c3e50;">Análisis</h3>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;">
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Idioma:</strong> {analisis["idioma"]}
                </div>
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Valoración:</strong> <span style="color: {color_valoracion}; font-weight: bold;">{valoracion.upper()}</span>
                </div>
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Envío en 96h:</strong> {analisis["envio_96h"]}
                </div>
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Embalaje dañado:</strong> {analisis["embalaje_danado"]}
                </div>
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Talla correcta:</strong> {analisis["talla_correcta"]}
                </div>
                <div style="background-color: white; padding: 10px; border-radius: 5px;">
                    <strong>Calidad materiales:</strong> {analisis["materiales_calidad"]}
                </div>
            </div>
            <div style="background-color: white; padding: 10px; border-radius: 5px; margin-top: 10px;">
                <strong>Tipo de uso:</strong> {analisis["tipo_uso"]}
            </div>
            <div style="background-color: white; padding: 10px; border-radius: 5px; margin-top: 10px;">
                <strong>Cumple expectativas:</strong> {analisis["cumple_expectativas"]}
            </div>
        </div>
        """

        email_cliente_html = ""
        if comunicaciones.get("email_cliente", ""):
            email_cliente_html = f"""
            <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
                <h3 style="margin-top: 0; color: #2c3e50;">Email para el Cliente</h3>
                <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif;">
                    {comunicaciones["email_cliente"]}
                </div>
            """

            if comunicaciones.get("email_cliente_traduccion", "") and analisis["idioma"] != "español":
                email_cliente_html += f"""
                <h4 style="margin-top: 15px; color: #2c3e50;">Traducción al Español:</h4>
                <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif; border-left: 3px solid #e31837;">
                    {comunicaciones["email_cliente_traduccion"]}
                </div>
                """

            email_cliente_html += "</div>"

        notificacion_interna_html = ""
        if comunicaciones.get("notificacion_interna", ""):
            notificacion_interna_html = f"""
            <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
                <h3 style="margin-top: 0; color: #2c3e50;">Notificación Interna</h3>
                <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif;">
                    {comunicaciones["notificacion_interna"]}
                </div>
            </div>
            """

        email_proveedor_html = ""
        if comunicaciones.get("email_proveedor", ""):
            email_proveedor_html = f"""
            <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
                <h3 style="margin-top: 0; color: #2c3e50;">Email para el Proveedor</h3>
                <div style="background-color: white; padding: 15px; border-radius: 5px; white-space: pre-wrap; font-family: Calibri, sans-serif;">
                    {comunicaciones["email_proveedor"]}
                </div>
            </div>
            """

        html_completo = f"""
        <div style="font-family: Calibri, sans-serif; max-width: 900px; margin: 0 auto;">
            <h2 style="color: #2c3e50; border-bottom: 2px solid #e31837; padding-bottom: 10px;">
                Resultado del Análisis y Comunicaciones Generadas
            </h2>

            {comentario_html}
            {analisis_html}
            {email_cliente_html}
            {notificacion_interna_html}
            {email_proveedor_html}

            <div style="text-align: right; font-size: 12px; color: #7f8c8d; margin-top: 20px;">
                Análisis generado el {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}
            </div>
        </div>
        """

        return html_completo

    except Exception as e:
        return f"""
        <div style="background-color: #fdedec; padding: 15px; border-radius: 8px; color: #c0392b;">
            <h3 style="margin-top: 0;">Error al generar resultado</h3>
            <p>{str(e)}</p>
            <pre>{comentario_original}</pre>
        </div>
        """


## **5. 🧠 Análisis automático del comentario con IA**

El núcleo funcional de esta herramienta es la capacidad de **analizar automáticamente comentarios de clientes** en múltiples idiomas, aplicando criterios específicos definidos por el equipo de calidad de KelceTS.

La función `analizar_comentario()` se encarga de:

- 🌍 Detectar el idioma del comentario
- 📋 Generar un análisis estructurado usando IA generativa
- ✅ Devolver una respuesta en formato JSON con siete aspectos clave:

  - Envío en 96 horas
  - Embalaje dañado
  - Talla correcta
  - Calidad de los materiales
  - Tipo de uso
  - Cumplimiento de expectativas
  - Idioma detectado

Internamente, esta función utiliza `call_ai_model()`, por lo que está preparada para funcionar con **OpenAI como modelo principal** y hacer fallback automático a **Gemini Pro** en caso de error.

> Este análisis es el primer paso del pipeline de IA, y alimentará las siguientes fases: clasificación de la valoración, generación de comunicaciones y visualización del resultado.


In [11]:
def analizar_comentario(texto: str) -> dict:
    """
    Analiza el comentario de un cliente usando el modelo de IA con fallback automático.
    Devuelve un diccionario con el bloque 'analisis'.
    """
    idioma = detectar_idioma(texto)

    prompt = f"""
    Analiza el siguiente comentario de un cliente de una tienda de zapatillas:

    "{texto}"

    El idioma del comentario es: {idioma}

    Por favor, analiza los siguientes aspectos y devuelve los resultados en formato JSON:
    1. ¿El envío se recibió en menos de 96h? (sí/no/no mencionado)
    2. ¿El embalaje estaba dañado? (sí/no/no mencionado)
    3. ¿La talla es correcta? (sí/no/no mencionado)
    4. ¿La calidad de los materiales es buena? (sí/no/parcialmente/no mencionado)
    5. ¿Qué tipo de uso da el cliente a las zapatillas? (descripción)
    6. ¿Cumple el producto las expectativas del cliente? (sí/no/parcialmente)

    Usa el siguiente formato exacto para la respuesta:
    {{
      "analisis": {{
        "idioma": "{idioma}",
        "envio_96h": "RESPUESTA",
        "embalaje_danado": "RESPUESTA",
        "talla_correcta": "RESPUESTA",
        "materiales_calidad": "RESPUESTA",
        "tipo_uso": "RESPUESTA",
        "cumple_expectativas": "RESPUESTA"
      }}
    }}
    """

    # Llamada al modelo con fallback (usando la función central)
    respuesta = call_ai_model(prompt, system_message="Eres un asistente especializado en análisis de comentarios de clientes.", model="gpt-3.5-turbo")

    # Intentar extraer el JSON del texto generado
    try:
        match = re.search(r'{[\s\S]*}', respuesta)
        if match:
            analisis = json.loads(match.group(0))["analisis"]
            return analisis
        else:
            raise ValueError("No se pudo extraer el bloque JSON del resultado.")
    except Exception as e:
        print(f"❌ Error al analizar: {str(e)}")
        return {
            "idioma": idioma,
            "envio_96h": "no mencionado",
            "embalaje_danado": "no mencionado",
            "talla_correcta": "no mencionado",
            "materiales_calidad": "no mencionado",
            "tipo_uso": "no mencionado",
            "cumple_expectativas": "no mencionado"
        }


## **6. 📊 Clasificación de la valoración del cliente**

Después de analizar el comentario del cliente, clasificamos su valoración como **positiva**, **neutra** o **negativa**. Esta clasificación será utilizada para decidir:

- El tipo de email que se genera para el cliente
- Si se notifica internamente a los equipos de calidad o logística
- Si se contacta con el proveedor para tomar medidas

La función `clasificar_valoracion()` aplica las siguientes reglas:

- ❌ Si hay al menos un "no" en los campos clave → valoración **negativa**
- ⚠️ Si hay al menos un "parcialmente" → valoración **neutra**
- ✅ Si no hay incidencias → valoración **positiva**

> Esta lógica permite automatizar la gestión sin perder el criterio humano original definido por los equipos internos.


In [12]:
def clasificar_valoracion(analisis: dict) -> str:
    """
    Clasifica la valoración del cliente como positiva, neutra o negativa.
    Basado en el bloque de 'analisis' generado por la IA.
    """
    if (
        analisis.get("talla_correcta") == "no" or
        analisis.get("materiales_calidad") == "no" or
        analisis.get("envio_96h") == "no" or
        analisis.get("embalaje_danado") == "sí"
    ):
        return "negativa"

    elif (
        analisis.get("materiales_calidad") == "parcialmente" or
        analisis.get("cumple_expectativas") == "parcialmente"
    ):
        return "neutra"

    else:
        return "positiva"


## **7. 🧠 Función principal de análisis desde Gradio**

La función `analizar_texto()` es el núcleo que conecta toda la lógica del asistente con la interfaz Gradio.

Permite a un agente del Call Center introducir un comentario manualmente y obtener:

- 🧠 Un análisis estructurado usando IA generativa (OpenAI o Gemini)
- 🟢 La valoración automática del cliente (positiva, negativa o neutra)
- 🔄 La traducción al español si el comentario está en otro idioma
- 💌 Las comunicaciones generadas automáticamente:
  - Email para el cliente
  - Notificación interna
  - Email para proveedor
- 🖼️ La presentación visual en HTML para mostrar el resultado en pantalla

Esta función integra de forma unificada:

- `detectar_idioma()`
- `translate_comment()`
- `analizar_comentario()`
- `clasificar_valoracion()`
- `generar_comunicaciones()`
- `generar_html_resultado()`


In [13]:
# Función principal para analizar comentario desde Gradio

def analizar_texto(texto: str) -> str:
    """
    Función principal que analiza un comentario completo desde Gradio
    y devuelve el resultado en HTML.
    """
    if not texto or texto.strip() == "":
        return "<div style='text-align:center; padding:20px; color:#e74c3c;'>Por favor ingresa un texto para analizar</div>"

    try:
        # 1 Detectar idioma
        idioma = detectar_idioma(texto)

        # 2️ Traducir comentario si no está en español
        traduccion = ""
        if idioma != "español":
            try:
                traduccion = translate_comment(texto, source_lang=idioma)
            except:
                traduccion = "No se pudo generar la traducción automática."

        # 3️ Analizar comentario con IA (OpenAI + Gemini)
        analisis = analizar_comentario(texto)

        # 4️ Clasificar valoración
        valoracion = clasificar_valoracion(analisis)

        # 5️ Preparar estructura base del resultado
        resultado = {
            "analisis": analisis,
            "valoracion": valoracion,
            "traduccion_comentario": traduccion,
            "comunicaciones": {}
        }

        # 6️ Generar comunicaciones con IA
        resultado["comunicaciones"] = generar_comunicaciones(resultado)

        # 7️ Mostrar resultado final en HTML
        return generar_html_resultado(resultado, texto)

    except Exception as e:
        return f"""
        <div style="background-color: #fdedec; padding: 15px; border-radius: 8px; color: #c0392b;">
            <h3 style="margin-top: 0;">Error al analizar texto</h3>
            <p>{str(e)}</p>
        </div>
        """


## **8. ✉️ Generación de comunicaciones personalizadas con IA**

Una vez analizado el comentario y determinada la valoración del cliente, el sistema debe generar de forma automática las diferentes comunicaciones necesarias para cada caso:

- 📨 Email personalizado para el cliente (en su idioma)
- 📬 Traducción del email al español (si aplica)
- 🛠️ Notificación interna para los equipos de calidad o logística
- 🧾 Email profesional para proveedores externos (si se requiere acción)

La función `generar_comunicaciones()`:

- Recibe como entrada el bloque `resultado` generado previamente
- Construye dinámicamente los `prompts` según el contexto del análisis
- Utiliza `call_ai_model()` para garantizar disponibilidad (OpenAI + Gemini)
- Devuelve un diccionario con las 3 o 4 comunicaciones clave, listas para ser mostradas o exportadas

> Esta modularización permite mantener el flujo limpio, profesional y escalable. Si en el futuro cambian los estilos, idiomas o formatos, solo será necesario actualizar esta función.


In [14]:
# Función para generar emails y notificaciones

def generar_comunicaciones(resultado: dict) -> dict:
    """
    Genera las comunicaciones necesarias en función del análisis y valoración.
    Devuelve un diccionario con:
    - email_cliente
    - email_cliente_traduccion
    - notificacion_interna
    - email_proveedor
    """

    analisis = resultado["analisis"]
    valoracion = resultado["valoracion"]
    idioma = analisis.get("idioma", "español")

    # --- 1. EMAIL AL CLIENTE ---
    if valoracion == "positiva":
        prompt_cliente = f"""
        Genera un email para un cliente satisfecho con unas zapatillas.

        Detalles:
        - Idioma: {idioma}
        - Talla correcta: {analisis.get("talla_correcta")}
        - Materiales: {analisis.get("materiales_calidad")}
        - Tipo de uso: {analisis.get("tipo_uso")}

        El email debe:
        - Estar en {idioma}
        - Ser amable y usar "tú"
        - Agradecer el comentario positivo
        - Incluir el email de contacto: atencionalcliente@kelcetssl.com
        - Terminar con "Un cordial saludo," y "KelceTS Team"
        """
    elif valoracion == "negativa":
        prompt_cliente = f"""
        Genera un email para un cliente insatisfecho con unas zapatillas.

        Problemas detectados:
        - Talla: {analisis.get("talla_correcta")}
        - Materiales: {analisis.get("materiales_calidad")}
        - Envío: {analisis.get("envio_96h")}
        - Embalaje: {analisis.get("embalaje_danado")}

        El email debe:
        - Estar en {idioma}
        - Usar tono empático y lenguaje informal ("tú")
        - Ofrecer disculpas y soluciones (descuento o reemplazo)
        - Incluir contacto: atencionalcliente@kelcetssl.com
        - Terminar con "Un cordial saludo," y "KelceTS Team"
        """
    else:  # neutra
        prompt_cliente = f"""
        Genera un email para un cliente con una experiencia mixta sobre unas zapatillas.

        Detalles del análisis:
        - Idioma: {idioma}
        - Materiales: {analisis.get("materiales_calidad")}
        - Uso reportado: {analisis.get("tipo_uso")}

        El email debe:
        - Estar en {idioma}
        - Reconocer tanto los aspectos positivos como los mejorables
        - Agradecer el feedback y reforzar nuestro compromiso con la calidad
        - Incluir contacto: atencionalcliente@kelcetssl.com
        - Terminar con "Un cordial saludo," y "KelceTS Team"
        """

    email_cliente = call_ai_model(prompt_cliente, system_message=f"Redacta un email para un cliente en {idioma}.", model="gpt-3.5-turbo")

    # --- 2. TRADUCCIÓN AL ESPAÑOL SI APLICA ---
    email_cliente_traduccion = ""
    if idioma != "español":
        try:
            email_cliente_traduccion = translate_comment(email_cliente, source_lang=idioma, target_lang="español")
        except Exception as e:
            email_cliente_traduccion = "No se pudo traducir el email al español."

    # --- 3. NOTIFICACIÓN INTERNA ---
    notificacion_interna = ""
    if valoracion != "positiva":
        prompt_interna = f"""
        Genera una notificación interna para el equipo de calidad/logística de una tienda de zapatillas.

        Detalles:
        - Valoración: {valoracion}
        - Talla: {analisis.get("talla_correcta")}
        - Materiales: {analisis.get("materiales_calidad")}
        - Envío: {analisis.get("envio_96h")}
        - Embalaje: {analisis.get("embalaje_danado")}
        - Uso reportado: {analisis.get("tipo_uso")}

        El mensaje debe:
        - Estar en español
        - Ser claro y conciso
        - Incluir acciones a tomar si aplica
        - Firmar como "Asistente IA de KelceTS S.L."
        """
        notificacion_interna = call_ai_model(prompt_interna, system_message="Eres un asistente interno para calidad/logística.", model="gpt-3.5-turbo")

    # --- 4. EMAIL A PROVEEDOR ---
    email_proveedor = ""
    if (
        analisis.get("talla_correcta") == "no" or
        analisis.get("materiales_calidad") == "no" or
        analisis.get("envio_96h") == "no" or
        analisis.get("embalaje_danado") == "sí"
    ):
        prompt_proveedor = f"""
        Redacta un email formal a un proveedor notificando un problema de calidad/logística detectado en un pedido de zapatillas.

        Detalles:
        - Talla incorrecta: {analisis.get("talla_correcta")}
        - Materiales defectuosos: {analisis.get("materiales_calidad")}
        - Retraso o problemas de envío: {analisis.get("envio_96h")}, Embalaje dañado: {analisis.get("embalaje_danado")}

        El email debe:
        - Ser profesional y cortés
        - Solicitar medidas correctivas en un plazo de 24-48h
        - Estar en español
        - Estar firmado por: Rodrigo Clemente, Director de Logística de KelceTS S.L.
        """
        email_proveedor = call_ai_model(prompt_proveedor, system_message="Eres un director de logística.", model="gpt-3.5-turbo")

    # --- Resultado final ---
    return {
        "email_cliente": email_cliente,
        "email_cliente_traduccion": email_cliente_traduccion,
        "notificacion_interna": notificacion_interna,
        "email_proveedor": email_proveedor
    }


## **9. 🧪 Procesamiento de ejemplos predefinidos sin coste de API**

La función `procesar_ejemplo()` permite a los agentes del Call Center analizar comentarios simulados sin necesidad de realizar llamadas a modelos de IA. Esto es especialmente útil para:

- Demostrar el funcionamiento completo de la aplicación Gradio
- Realizar pruebas sin consumir créditos de API
- Validar el diseño de la interfaz sin depender de la conectividad

Esta función:

- Toma un índice (`ejemplo_idx`) y recupera el comentario correspondiente de la lista `EJEMPLOS_COMENTARIOS`
- Usa la respuesta estructurada simulada desde `RESPUESTAS_SIMULADAS`
- Realiza la **traducción automática del comentario si no está en español**, utilizando la función `translate_comment()`
- Muestra el resultado en formato HTML con la función `generar_html_resultado()`

> Al no usar llamadas en tiempo real a OpenAI ni Gemini, esta función es ideal para formación, testeo y demostraciones sin riesgo ni latencia.


In [15]:
def procesar_ejemplo(ejemplo_idx: int):
    """Procesa un ejemplo de la lista predefinida para mostrarlo sin coste de API"""
    try:
        idx = int(ejemplo_idx)
        if idx < 0 or idx >= len(EJEMPLOS_COMENTARIOS):
            return "<div style='text-align:center; padding:20px; color:#e74c3c;'>Índice de ejemplo inválido</div>"

        comentario = EJEMPLOS_COMENTARIOS[idx]
        resultado = RESPUESTAS_SIMULADAS[idx].copy()

        # Añadir traducción del comentario si no está en español
        idioma = resultado["analisis"].get("idioma", "desconocido")
        if idioma != "español":
            try:
                traduccion = translate_comment(comentario, source_lang=idioma)
                resultado["traduccion_comentario"] = traduccion
            except Exception as e:
                print(f"Error al traducir comentario: {e}")
                resultado["traduccion_comentario"] = "No se pudo traducir."

        return generar_html_resultado(resultado, comentario)

    except Exception as e:
        return f"""
        <div style="background-color: #fdedec; padding: 15px; border-radius: 8px; color: #c33333;">
            <h3 style="margin-top: 0;">Error al procesar ejemplo</h3>
            <p>{str(e)}</p>
        </div>
        """


## **10. 🧩 Interfaz Gradio para agentes del Call Center de KelceTS**

La aplicación Gradio permite a los agentes del Call Center de KelceTS interactuar con el sistema de análisis de comentarios de forma visual e intuitiva.

Esta interfaz incluye:

- 🧪 Una pestaña con ejemplos predefinidos para pruebas sin coste de API
- 💬 Una pestaña donde el agente puede escribir o pegar un comentario real
- 🎯 Salida en HTML con:
  - Análisis estructurado
  - Valoración del cliente
  - Traducción automática (si aplica)
  - Comunicaciones generadas por IA

> El objetivo es ofrecer una herramienta realista, robusta y operativa para su uso inmediato en entornos reales.

---


### ***10.1 ✅ Guía de Uso para Agentes del Call Center***

Esta herramienta ha sido diseñada para integrarse fácilmente en el día a día del Call Center de **KelceTS S.L.**, y puede ser utilizada por cualquier agente con conocimientos básicos de atención al cliente.

---


### ***10.2 🖥️ ¿Cómo se usa esta aplicación?***

La interfaz incluye dos pestañas claramente diferenciadas:

1. **📚 Ejemplos Predefinidos**  
   Permite analizar comentarios simulados ya cargados, ideales para formación, demostraciones o pruebas sin consumo de APIs.

2. **✍️ Comentario Manual**  
   Puedes pegar aquí cualquier comentario real recibido por email, redes sociales u otros canales.  
   El sistema se encarga automáticamente de:

   - Detectar el idioma original del cliente  
   - Traducir al español (si es necesario)  
   - Analizar el contenido (envío, embalaje, talla, calidad, uso, expectativas)  
   - Clasificar la valoración como positiva, neutra o negativa  
   - Generar automáticamente:
     - 📩 Email personalizado al cliente
     - 🛠️ Notificación interna (si aplica)
     - 📦 Email al proveedor (si aplica)

---


### ***10.3 📤 Resultado en pantalla***

El resultado del análisis aparece directamente en pantalla en formato HTML profesional, listo para:

- Copiar y pegar en respuestas
- Validar y editar si se desea
- Enviar desde la herramienta oficial del Call Center

> El sistema seleccionará automáticamente entre **OpenAI** y **Gemini** en función de disponibilidad, garantizando fiabilidad en entornos reales.


In [ ]:
# Interfaz Gradio con branding y estilo KelceTS

with gr.Blocks(title="Asistente IA Call Center – KelceTS") as interfaz:

    # ✅ Cabecera compacta con logo + texto en blanco + fondo rojo Chiefs
    gr.HTML("""
    <div style="display: flex; align-items: center; justify-content: flex-start; background-color: #e31837; padding: 10px 20px; border-radius: 10px;">
        <img src='https://raw.githubusercontent.com/AraceliFradejas/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/main/data/Kelce%20TS%20LOGO.png'
             alt='Logo KelceTS' style='height: 70px; margin-right: 20px;' />
        <div style="color: white;">
            <h1 style="margin: 0; font-size: 22px; color: white;">Asistente de IA para el Call Center de KelceTS</h1>
            <p style="margin: 4px 0 0; font-size: 14px; color: white;">Automatiza, responde y conquista la atención al cliente con inteligencia ⚡</p>
        </div>
    </div>
    """)

    # ✅ Tabs para modos de uso
    with gr.Tabs():
        with gr.TabItem("👟 Ejemplos Predefinidos"):
            gr.Markdown("Selecciona un comentario de ejemplo para analizarlo sin coste de API:")

            selector = gr.Radio(
                choices=[f"Ejemplo {i+1}: {c[:60]}..." for i, c in enumerate(EJEMPLOS_COMENTARIOS)],
                label="Selecciona un comentario",
                type="index"
            )

            boton_analizar_ejemplo = gr.Button(
                "🔍 Analizar Ejemplo",
                elem_id="btn_ejemplo"
            )
            salida_ejemplo = gr.HTML()

            boton_analizar_ejemplo.click(fn=procesar_ejemplo, inputs=[selector], outputs=[salida_ejemplo])

        with gr.TabItem("📝 Comentario Manual"):
            gr.Markdown("Introduce un comentario de cliente para analizar automáticamente:")

            entrada_texto = gr.Textbox(
                label="Comentario del Cliente",
                placeholder="Escribe aquí el comentario recibido por email, redes, etc.",
                lines=5
            )

            boton_analizar_texto = gr.Button(
                "🤖 Analizar Comentario",
                elem_id="btn_manual"
            )
            salida_texto = gr.HTML()

            boton_analizar_texto.click(fn=analizar_texto, inputs=[entrada_texto], outputs=[salida_texto])

    # ✅ Cargar primer ejemplo al inicio
    interfaz.load(fn=lambda: procesar_ejemplo(0), inputs=None, outputs=salida_ejemplo)

    # ✅ Estilos personalizados para botones estilo Chiefs
    gr.HTML("""
    <style>
    #btn_ejemplo, #btn_manual {
        background-color: #e31837 !important;
        color: #ffb81c !important;
        font-weight: bold;
        border-radius: 8px;
        border: none;
        padding: 10px 20px;
    }
    #btn_ejemplo:hover, #btn_manual:hover {
        background-color: #b3001b !important;
        color: white !important;
    }
    </style>
    """)
    # ✅ Footer visual con branding
    gr.HTML("""
    <div style="text-align: center; margin-top: 30px; padding: 20px 10px; font-family: Calibri, sans-serif; font-size: 14px; color: #555;">
        <hr style="border: none; height: 2px; background-color: #e31837; margin-bottom: 15px;">

        <img src='https://raw.githubusercontent.com/AraceliFradejas/Capstone-Project---Desarrollador10X-IIA---Araceli-Fradejas/main/data/Kelce%20TS%20LOGO.png'
             alt='Logo KelceTS' style='height: 50px; margin-bottom: 10px;' />

        <p><strong>Asistente de IA para un call center desarrollado por Araceli Fradejas Muñoz, Abril 2025</strong></p>
        <p>Curso Desarrollador10X realizada en el <a href="https://iia.es/" target="_blank" style="color: #e31837; font-weight: bold;">Instituto de Inteligencia Artificial</a></p>
        <p style="font-size: 13px; color: #999;">KelceTS S.L. – Proyecto académico ficticio como Capstone Project de IA generativa</p>
    </div>
    """)

# 🚀 Lanzamiento
interfaz.launch(share=True, debug=True)



Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://08e95050f7815d5f61.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔵 Llamando a OpenAI...
✅ Respuesta generada por OpenAI.


# **13. 📘 Agradecimientos y Cierre**

Gracias al equipo docente del Institututo de Inteligencia Artificial.  
Ha sido toda una experiencia súper enriquecedoara formarme con vosotros.

A todos los que me puedan leer os recomiendo formaros con ellos.  
Os dejo aquí su link: https://iia.es/

¡Muchísimas gracias! 😍

**Araceli Fradejas Muñoz**
